# DataFrame vs SQL

In [1]:
import pandas as pd
import numpy as np

In [4]:
emp = pd.read_csv("./lec08_emp.csv", parse_dates=['HIREDATE'])
emp.head(3)
# 컬럼이름 소문자변경
emp.columns = emp.columns.str.lower()

## Select문

ORACLE 문법  
SELECT * FROM EMP;  

DataFrame 문법  
<font hidden>
    ```emp```
<font>

ORACLE 문법   
SELECT ENAME FROM EMP;  

DataFrame 문법  
<font hidden>
```emp['ename']```    
<font>

ORACLE 문법  
SELECT ENAME, EMPNO FROM EMP;  

DataFrame 문법  
<font hidden>
    `emp[  ['empno', 'ename']  ]`
<font>

ORACLE 문법  
SELECT EMPNO, ENAME  
FROM EMP  
WHERE ROWNUM <= 3;

DataFrame 문법  
<font hidden>
    `emp.loc[0:2, ['empno','ename']]`
<font>

### iloc, loc 문법

### 값 뽑아보기 연습

In [3]:
# emp.iloc[헹]
# emp.iloc[3]               # 단일값
# emp.iloc[  [0,3,5]  ]     # 여러값
# emp.iloc[ 0: 3]           # 슬라이스

# emp.iloc[행,열]
# emp.iloc[0, 1]            # 단일값
# emp.iloc[ 0, [0,3]]       # 여러값
# emp.iloc[ 0:3, 2:4]         # 슬라이스

In [4]:
# emp.loc[행값, 열값]                     **주의 값임!
# emp.loc[0, 'ename']                    # 단일값, 단일값
# emp.loc[[1,2,3], ['empno','ename']]    # 여러값, 여러값
# emp.loc[ 0:3, 'empno':'mgr']           # 슬라이싱, 슬라이싱(** 여기는 미만이 아님(3, mgr포함) **)

## Where 조건

<hr>

<font size=5 style=bold >사용법</font>  
df[조건]  
df[조건][컬럼]  
df[컬럼][조건]

ORACLE 문법  
SELECT * FROM EMP WHERE DEPTNO = 10;

DataFrame 문법  
<font hidden>
    `emp[  emp['deptno'] ==10  ]`
<font>

### 단일조건
ORACLE 문법  
SELECT EMPNO, ENAME FROM EMP WHERE DEPTNO = 10;

DataFrame 문법
<font hidden>
    `emp[ emp['deptno'] == 10  ][  ['empno', 'ename']   ]`  
<font>


**내가생각한 문제   iloc에 조건을 걸수있을까 iloc으로 df로 뽑고  **   
emp.iloc[1:5, :][ emp['deptno'] == 20]['ename']

### 멀티조건
ORACLE 문법  
SELECT EMPNO, ENAME, SAL FROM EMP WHERE DEPTNO = 10 AND SAL > 2000;

DataFrame 문법  
<font hidden>
    `emp[ ['empno', 'ename'] ][ (emp['deptno'] == 10) & (emp['sal'] > 2000) ]`
<font>

## Subquery 비추

### 문제1  
사번이 7369인 사원의 직업과 같은 사원들의 이름, 직업 출력  
ORACLE 문법  
SELECT ENAME, JOB  
FROM EMP  
WHERE JOB =  
(SELECT JOB  
FROM EMP  
WHERE EMPNO = 7369);

DataFrame 문법  
<font hidden>
    `emp['job'][(emp['empno'] == 7369)].values[0]`  
    `job7369 = emp['job'][(emp['empno'] == 7369)].values[0]`  
    `emp[ ['ename', 'job'] ][  emp['job']  ==  job7369  ]`      
<font>


### 문제2
10번부서 사람들의 직업과 같은 사원의 사원들의 이름, 직업 출력  
ORACLE 문법  
SELECT ENAME, JOB  
FROM EMP  
WHERE JOB IN (SELECT JOB FROM EMP WHERE DEPTNO = 10);

DataFrame 문법  
<font hidden>
    `job10_arr = emp['job'][ emp['deptno'] == 10 ].values`  
    `emp[['ename', 'job']][ emp['job'].isin( job10_arr )   ]`
<font>

## 컬럼연산

### 문제1
합치기
ORACLE 문법  
SELECT JOB||ENAME FROM EMP  
SELECT CONCAT(JOB, ENAME) FROM EMP  

DataFrame 문법  
<font hidden>
    emp['job'] + emp['ename']
<font>

### 문제2
연봉뽑기  
ORACLE 문법   
SELECT SAL*12 AS ASAL FROM EMP;  

DataFrame 문법  
<font hidden>
    `emp['asal'] = emp['sal'] * 12`
<font>

## 결측처리
* df.dropna()
* df['comm'].fillna(0) -- nvl(comm, 0)
* emp['comm'].isna() --------

### 문제 1
ORACLE 문법  
SELECT COMM, NVL(COMM, 0) AS COMM2 FROM EMP;  

DataFrame 문법  
<font hidden>
`emp['comm2'] = emp['comm'].fillna(0)`  
`emp[['comm', 'comm2']]`
<font>

## GROUP 조건

### 문제1
각부서별 사원수 출력  
ORACLE 문법  
SELECT DEPTNO, COUNT(1) AS 사원수  
FROM EMP  
GROUP BY DEPTNO  
ORDER BY DEPTNO;  

DataFrame 문법  
<font hidden>
    emp.groupby('deptno')['deptno'].count()
<font>

### 문제2
각부서별 급여 합  
ORACLE 문법  
SELECT DEPTNO, sum(sal)
FROM EMP  
GROUP BY DEPTNO;  

DataFrame 문법  
<font hidden>
    emp.groupby('deptno')['sal'].sum()
<font>

### 문제3
부서별 최대 최소 급여  
ORACLE 문법  
SELECT DEPTNO, MAX(SAL), MIN(SAL)  
FROM EMP  
GROUP BY DEPTNO;

DataFrame 문법  
<font hidden>
    emp.groupby('deptno')['sal'].min()
    emp.groupby("deptno").agg(  {"sal":"max", "comm" : "min"}   )
    emp.groupby("deptno").agg(  {"sal":"max", "sal" : "min"}   ) --에러 (key가 같아서 오류)
    emp.groupby("deptno")["sal"].agg(  ["max", "min"]   )
<font>

### 문제4
각 부서별 급여 합, 최대, 최소, 평균  
ORACLE 문법  
SELECT DEPTNO, SUM(SAL), MAX(SAL), MIN(SAL), ROUND(AVG(SAL),0), count(1)  
FROM EMP  
GROUP BY DEPTNO;  

DataFrame 문법  
<font hidden>
    emp.groupby('deptno')["sal"].agg( ["sum", "max", "min", "mean", 'count', 'std', 'var'] )
<font>

## 정렬

### 문제1
ORACLE 문법  
SELECT SAL FROM EMP ORDER BY SAL DESC;

DataFrame 문법  
<font hidden>
    emp['sal'].sort_values(ascending=True)
<font>

### 문제2
ORACLE 문법  
SELECT EMPNO, ENAME, SAL FROM EMP WHERE DEPTNO = 20 ORDER BY ENAME ASC;

DataFrame 문법  
<font hidden>
    e = emp[  ['empno', 'ename', 'sal'] ][ emp['deptno'] == 20]
    e.sort_values(by=['ename'], ascending=True)
<font>

### 문제3
ORACLE 문법  
SELECT EMPNO, DEPTNO, SAL FROM EMP ORDER BY DEPTNO ASC, SAL DESC;

DataFrame 문법  
<font hidden>
    emp[['empno', 'deptno', 'sal']].sort_values(by=["deptno","sal"], ascending=[True, False])
<font>

In [5]:
# 인덱스로 소트하기
# emp.sort_index(ascending=False)

## UPDATE
at, iat은 단일값 변경만 가능 (여러개는 불가)  
* df[기존컬럼] = 값   ---> 수정  
* df[신규컬럼] = 값   ---> 신규컬럼생성  
df.at[행값, 열값] = 변경값  
df.iat[행idx, 열idx] = 변경값

여러개를 할 때는 loc, iloc을 사용한다.  
df.loc[행값, 열값] = 변경값  
df.loc[행idx, 열idx] = 변경값

### 문제1
ORACLE 문법  
UPDATE EMP  
SET COMM = 999;

DataFrame 문법  
<font hidden>
    emp['comm'] = 99999
<font>

### 문제2
ORACLE 문법  
UPDATE EMP
SET ENAME = 'SSSMITHSS'
WHERE 첫번째데이터

DataFrame 문법  
<font hidden>
emp.at[0, "ename"] 'SSSMITHSS'
emp.at[0, 1] 'SSSMITHSS'

<font>

## np.where() vs df.where()

방법 2가지 (   
np.where(조건절, 참값, 거짓값),    -- decode(deptno, 10, '십', '그외')   
df[컬럼A].where(조건문, 거짓값))   -- 조건만족하면 컬럼A, 그렇지않으면 거짓값  

### 문제1
ORACLE 문법  
UPDATE EMP
SET JOB = 'SALES10'
WHERE DEPTNO = 10;

DataFrame 문법  
<font hidden>
    emp['job'] = np.where(emp['deptno'] == 10, 'SALES10', emp['job']).sort_values(by='deptno')
<font>

### 문제2
ORACLE 문법  
UPDATE EMP
SET JOB = 'SALES10'
WHERE DEPTNO = 10;


DataFrame 문법  
<font hidden>
    emp['job'] = emp['job'].where(emp['deptno'] != 10, 'SALES1111')
<font>

## DELETE
* DML : delete from emp; -- 데이터 삭제
* DDL : drop table emp;  -- 테이블 삭제
* drop(**인덱스값**, axis= 0) .... dropna()   데이터행 삭제(가로지우기)
* drop(**컬럼명**, axis=1) .... dropna()      컬럼삭제 (세로지우기)
* del emp                -- 데이터프레임 삭제 drop table과 유사

### 문제1
ORACLE 문법  
DELETE FROM EMP WHERE ROWNUM ==1

DataFrame 문법  
<font hidden>
    emp = emp.drop(1, axis=0)
<font>

### 문제2
ORACLE 문법  
DELETE FROM EMP


DataFrame 문법  
<font hidden>
    emp = emp.drop('ename', axis=1)
<font>